# Generative AI Tools: A Deep Dive into LangChain and yFinance
**Author**: Mahdy Mokhtari  
**Date**: 6 Azar 1404 
**Course**: GenAI - Tools

## Overview
This notebook explores the integration of the **LangChain** framework with **yFinance**, a Python library used to fetch real-time stock prices. The aim of this project is to demonstrate how to build an interactive agent using these tools that can fetch stock prices dynamically and respond to user queries.

### Key Technologies Used:
1. **LangChain Framework**:  
   LangChain is a powerful framework that allows the integration of external tools and APIs into AI models. It facilitates the creation of custom agents that can combine the power of various Python libraries and language models (LLMs). In this project, LangChain is used to wrap Python functions and expose them as tools that can be invoked by AI models.

2. **yFinance**:  
   `yFinance` is a library that simplifies access to financial data from Yahoo Finance. This tool is leveraged in the notebook to retrieve real-time stock prices, offering an easy way to access and use historical market data for financial analysis.

3. **Python Programming**:  
   The core of the project is based on Python, a versatile language widely used in data science, AI, and finance. The Python code is designed to interact with the Yahoo Finance API via `yFinance` and to integrate seamlessly with the LangChain framework to facilitate natural language-based querying.


### Function to get Stock Price

In [1]:
# !pip install yfinance


In [2]:
# !pip install -U langchain-ollama

In [3]:
import yfinance as yf
from langchain_core.tools import tool
from pydantic import BaseModel, Field


class GetStockInput(BaseModel):
    """Schema for stock price input."""
    ticker_symbol: str = Field(..., description="The stock ticker symbol (e.g., 'AAPL', 'GOOGL')")    

@tool(args_schema=GetStockInput, return_direct=True, description="Fetches the latest closing stock price for a given ticker symbol using yFinance.")
def get_stock_price(ticker_symbol: str) -> str:
    data = yf.Ticker(ticker_symbol).history(period="1d")
    price = data['Close'].iloc[-1]
    return f"The current price of {ticker_symbol.upper()} is ${price:.2f}"




class AddInput(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

@tool("add-tool", args_schema=AddInput, return_direct=True, description="Adds two integers together and returns the result.")
def add(a: int, b: int) -> int:
    return a + b


tools = [get_stock_price, add]

In [7]:
import langchain
print(langchain.__version__)


1.0.3


In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="gpt2")   # free LLM

# llm_with_tools = llm.bind_tools(tools)    


ModuleNotFoundError: No module named 'langchain.prompts'

In [ ]:

llm.invoke("""135165435131313322323130130123 + 1233255552224463222111111""")
#correct response = 13602805

In [ ]:
llm.invoke("""135165435131313322323130130123 + 1233255552224463222111111""", tools=tools).content
# .tool_calls

### The Query

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages.tool_message import ToolMessage



SYSTEM_PROMPT = """
You are a helpful, precise, and reliable AI assistant.

Your goals:
1. Answer the user's query as accurately, concisely, and factually as possible.
2. If a tool is available AND it is appropriate for the user's request,
   you MUST call the tool using the correct function name and JSON arguments.
3. If no tool is relevant, answer directly in natural language.
4. If multiple tools could be relevant, choose the single best one.
5. Before calling a tool, verify that the required arguments are present and valid.
6. Never hallucinate tool names, arguments, or fields.

How you should behave when deciding between answering or calling a tool:
- If the user asks for **retrieval, calculation, an action, or external data**, use a tool.
- If the user asks a **knowledge, reasoning, or explanation question**, respond normally.
- When calling a tool, output ONLY the tool call in the required JSON format—no extra text.

If extra context is provided below, use it when helpful.

# Additional Context:
{context}

"""
context = "The user may ask for stock prices or to add two integers."


USER_PROMPT = """
# User Query:
{query}
"""

query = "What is the current stock price of Amazon?"


prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", USER_PROMPT),
])



# messages is just for the chat history
messages = [prompt.format_messages(context=context, query=query)[0]]

ai_msg = llm.invoke(messages, tools=tools)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    print(tool_call)
    print("Funciton/Tool name:", tool_call["name"])
    selected_tool = {"add": add, "get_stock_price": get_stock_price, 'add-tool': add, "get_stock_price-tool": get_stock_price}[tool_call["name"].lower()]

    print("tool args:", tool_call["args"])
    tool_output = selected_tool.invoke(tool_call["args"])

    print("tool output:", tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))
